# Customer churn Modelling
- This notebook if you are reading than you're familiar with kaggle and datascience so not much explanation is there but     the code will be self-explanatory for you.
- This notebook is to give idea how to start a notebook when machine learning comes into picture.
- I highly recommend first try it with by yourself than see these codes if some problem persists or simply to know what       your fellow data aspirant had done it may possible you would be doing better ;).
- If you feel good by scrolling it or you find any thing whether good or bad just comment it and you can upvote it also       "sakht launde ho to kr dena upvote".

# You will definitely like this outlier remover function:)

# All the Best

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

In [ ]:
df = pd.read_csv('../input/customer-churn-prediction-with-notebook/Churn History Dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#checking why phone number is float
df['phone number'] = df['phone number'].apply(lambda x: int(x.replace('-','')))
df['phone number']   


In [ ]:
df.dropna(inplace = True)
df.info()

In [ ]:
sns.countplot(df['state'])

In [ ]:
plt.subplot(1,2,1)
sns.countplot(df['international plan'])
plt.subplot(1,2,2)
sns.countplot(df['voice mail plan'])

In [ ]:
sns.countplot(df['Churn Indicator'])

In [ ]:
len(df[df['Churn Indicator'] == ' False.'])

In [ ]:
#checking data balance
l = len(df[df['Churn Indicator'] == ' True.'])
r = len(df[df['Churn Indicator'] == ' False.'])
print('% of trues in data is {}'.format(round((l/len(df))*100,2)))
print('% of falses in data is {}'.format(round((r/len(df))*100,2)))

In [ ]:
#plt.figure(figsize = (25,25))
#sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr(),annot = True, cmap = 'coolwarm')

In [ ]:
#dropping some feature due to correlation
df.drop(['phone number','total day charge','total eve charge','total intl charge',
         'total intl minutes','total night calls','total night charge'],axis = 1,inplace =True)
new_df = df
columns_for_test = new_df.columns

In [ ]:
sns.heatmap(new_df.corr(),annot = True, cmap = 'coolwarm')

In [ ]:
new_df.info()

In [ ]:
#seeing outliers
i = 0
outlier_col_list = []
plt.subplots_adjust(hspace = 3.0)
for item in new_df.columns:
    if new_df[item].dtypes != 'object':
        outlier_col_list.append(item)
        plt.subplot(10,2,i+1)
        sns.boxplot(df[item])
        plt.title(item,fontsize = 20)
        i+=1

In [ ]:
#outliers:
#account length
#number vmail messages(only 1 outlier )
#total day minutes
#total day calls (only 2 outlier)
#total eve minutes
#

In [ ]:

#finding outliers:
#interquartile method
def outlier_detect(df_col):
    q75, q25 = np.percentile(df_col, [75 ,25])
    iqr = q75 - q25
    min_val = q25 - (iqr*1.5)
    max_val = q75 + (iqr*1.5)
    return min_val, max_val

def remove_outliers(df_col,a,b):
    if df_col<a and df_col>b:
        return np.nan
    else:
        return df_col

#start removing outliers
from scipy.stats.mstats import winsorize
plt.figure(figsize=(18,6))

def outlier_treatment(new_df,target_col,pred_col='Churn Indicator'):
    
    l = len(new_df[new_df[pred_col] == ' True.'][target_col])
    r = len(new_df[new_df[pred_col] == ' False.'][target_col])

    print('before removing outliers % of trues in data is {}'.format(round((l/len(df))*100,2)))
    print('before removing outliers % of falses in data is {}'.format(round((r/len(df))*100,2)))

    a,b = outlier_detect(df[target_col])

    df[target_col] = df[(df[target_col] >= a) & (df[target_col] <= b) ][target_col]

    l = len(new_df[new_df[pred_col] == ' True.'][target_col].dropna())
    r = len(new_df[new_df[pred_col] == ' False.'][target_col].dropna())
    print('-------------------------------------------------------------------------------')
    print('after removing outliers % of trues in data is {}'.format(round((l/(l+r))*100,2)))
    print('after removing outliers % of falses in data is {}'.format(round((r/(l+r))*100,2)))

    plt.subplot(1,2,1)
    original_data_column = new_df[target_col]
    plt.boxplot(new_df[target_col])
    plt.title("original_data_column")

    plt.subplot(1,2,2)
    plt.boxplot(new_df[target_col].dropna())                           
    plt.title("removed_outliers")

    plt.show()

In [ ]:

for item in outlier_col_list:
    outlier_treatment(new_df,item)

In [ ]:
i = 0
plt.subplots_adjust(hspace = 3.0)
for item in new_df.columns:
    if new_df[item].dtypes != 'object':
        plt.subplot(9,2,i+1)
        sns.boxplot(df[item])
        plt.title(item,fontsize = 20)
        i+=1

In [ ]:
#checking data balance
l = len(new_df[new_df['Churn Indicator'] == ' True.'])
r = len(new_df[new_df['Churn Indicator'] == ' False.'])
print('% of trues in data is {}'.format(round((l/len(df))*100,2)))
print('% of falses in data is {}'.format(round((r/len(df))*100,2)))

In [ ]:
new_df.info()

In [ ]:
for item in new_df.columns:
    if new_df[item].dtypes != 'object':
        new_df.fillna(new_df[item].mean(),inplace = True)

In [ ]:
df = new_df

In [ ]:
s = df.dtypes == 'object'
object_cols = s[s].index
print(object_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for cols in object_cols:
    df[cols] = encoder.fit_transform(df[cols])
df = df.astype(float)

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

y = df['Churn Indicator']
df.drop('Churn Indicator',axis =1, inplace =True)
columns_to_be_keep = (df.columns)
print(df.columns)
ros = RandomOverSampler()
df = scaler.fit_transform(df)
new_X_train,new_y = ros.fit_sample(df,y)

X_train,X_val,y_train,y_val = train_test_split(new_X_train,new_y,test_size = 0.3,random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

n = 100
n_list_train = []
n_list_val = []
range = [100,200,300,400,500,600,700,800,900,1000]
for i in [100,200,300,400,500,600,700,800,900,1000]:
    model = RandomForestClassifier(n_estimators = i)
    model.fit(X_train,y_train)
    pred = model.predict(X_val)
    val_score = accuracy_score(y_val,pred)
    n_list_val.append(val_score)
    pred_x = model.predict(X_train)
    train_score = accuracy_score(y_train,pred_x)
    n_list_train.append(train_score)



In [ ]:
plt.figure(figsize = (15,5))
plt.plot([100,200,300,400,500,600,700,800,900,1000],n_list_train,label = 'train accuracy',marker = 'o')
plt.plot([100,200,300,400,500,600,700,800,900,1000],n_list_val,label = 'validation accuracy',marker = 'o')
plt.xlabel('n_estimators')
plt.ylabel('accuracy')
plt.legend()
plt.title('Random Forest Classifier')
plt.show()

In [ ]:
model = RandomForestClassifier(n_estimators = 200)
model.fit(X_train,y_train)
pred = model.predict(X_val)
print(confusion_matrix(y_val,pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,cross_val_score,KFold,RandomizedSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
def fit_model(model,X_train, y_train, X_val, y_val, cross_val=False, cv_folds=5):
    
    model.fit(X_train,y_train)
    #Predict values:
    training_predictions = model.predict(X_train)
    validation_predictions = model.predict(X_val)
    model_report(X_train,y_train, X_val,y_val, training_predictions, validation_predictions)

    if cross_val:
        evaluate_cross_validation(model, X_train, y_train, cv_folds)
        
def model_report(X_train,y_train, X_val,y_val, training_predictions, validation_predictions):
    #Print model report:
    print("\nModel Report")
    print("Training")
    print("accuracy score : {}".format(accuracy_score(y_train, training_predictions)))
    #print("Root Mean Squared Error : {}".format(np.sqrt(mean_squared_error(y_train, training_predictions))))
    #print("R2 Score: {}".format(r2_score(y_train, training_predictions)))
    print("\n")
    print("Validation")
    print("accuracy score  : {}".format(accuracy_score(y_val, validation_predictions)))
    #print("Root Mean Squared Error : {}".format(np.sqrt(mean_squared_error(y_val, validation_predictions))))
    #print("R2 Score: {}".format(r2_score(y_val, validation_predictions)))
    
def evaluate_cross_validation(model, X_train, y_train, K):
    
    cv = KFold(n_splits=K, shuffle=True, random_state=2)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring = 'accuracy')
    print()
    print(scores)
    print("Mean score: {} max is {} and min {}".format(scores.mean(), max(scores), min(scores)))   
    
def random_search(model, param_grid, X_train, y_train, X_val, y_val, cv):
    
    random_search = RandomizedSearchCV(model, param_grid, scoring="accuracy", n_jobs=2, cv=cv)
    random_result = random_search.fit(X_train, y_train)

    # summarize results
    print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
    means = random_result.cv_results_['mean_test_score']
    stds = random_result.cv_results_['std_test_score']
    params = random_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    validation_predictions = random_result.predict(X_val)

    print("Validation")
    print("accuracy score : {}".format(accuracy_score(y_val, validation_predictions)))
    #print("Root Mean Squared Error : {}".format(np.sqrt(mean_squared_error(y_val, validation_predictions))))
    #print("R2 Score: {}".format(r2_score(y_val, validation_predictions)))

In [ ]:
model = RandomForestClassifier(n_estimators = 400)
fit_model(model,X_train,y_train,X_val,y_val,cross_val=True)

In [ ]:
df_test = pd.read_csv('../input/customer-churn-prediction-with-notebook/Churn Test Dataset.csv')
#columns_to_be_keep.append('Churn Indicator')
df_test = df_test[columns_for_test]

In [ ]:
s = df_test.dtypes == 'object'
object_cols = s[s].index
for col in object_cols:
    df_test[col] = encoder.fit_transform(df_test[col])
y_test = df_test['Churn Indicator']
df_test.drop('Churn Indicator',axis = 1,inplace = True)
df_test.astype(float)
df_test = scaler.transform(df_test)

pred = model.predict(df_test)
score = accuracy_score(y_test.astype(float),pred)
print(score)
print(confusion_matrix(y_test, pred))

In [ ]:
columns = columns_to_be_keep

importances = model.feature_importances_
weights = pd.Series(importances,
                 index=columns)
weights.sort_values()[:].plot(kind = 'barh')

In [ ]:
#decsion tree

#model=DecisionTreeClassifier(random_state=n)
n_list_train = []
n_list_val = []

for i in [5,10,15,20,25]:
    tree_model=DecisionTreeClassifier(random_state=i)
    tree_model.fit(X_train,y_train)
    pred = tree_model.predict(X_val)
    val_score = accuracy_score(y_val,pred)
    n_list_val.append(val_score)
    pred_x = tree_model.predict(X_train)
    train_score = accuracy_score(y_train,pred_x)
    n_list_train.append(train_score)


In [ ]:
plt.figure(figsize = (15,5))
plt.plot([5,10,15,20,25],n_list_train,label = 'train accuracy',marker = 'o')
plt.plot([5,10,15,20,25],n_list_val,label = 'validation accuracy',marker = 'o')
plt.xlabel('n_estimators')
plt.ylabel('accuracy')
plt.legend(loc='upper right')
plt.title('Random Forest Classifier')
plt.show()

In [ ]:
tree_model=DecisionTreeClassifier(random_state=20)
fit_model(tree_model,X_train,y_train,X_val,y_val,cross_val=True)

In [ ]:
columns = columns_to_be_keep
importances = tree_model.feature_importances_
weights = pd.Series(importances,
                 index=columns)
weights.sort_values()[-10:].plot(kind = 'barh')

In [ ]:
'''max_depth = [5, 10, 15, 20]
max_features = ['log2', 'sqrt', 'auto']
min_samples_leaf = [10, 50, 70, 100]
param_grid = dict(max_depth=max_depth, max_features=max_features, 
                  min_samples_leaf=min_samples_leaf)
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

random_search(DecisionTreeRegressor(random_state=15), param_grid, X_train, y_train, X_val, y_val, kfold)'''

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
#decsion tree

#model=DecisionTreeClassifier(random_state=n)
n_list_train = []
n_list_val = []

for i in [0.0001,0.001,0.01]:
    log_model=LogisticRegression(C = i)
    log_model.fit(X_train,y_train)
    pred = log_model.predict(X_val)
    val_score = accuracy_score(y_val,pred)
    n_list_val.append(val_score)
    pred_x = log_model.predict(X_train)
    train_score = accuracy_score(y_train,pred_x)
    n_list_train.append(train_score)

In [ ]:
plt.figure(figsize = (15,5))
plt.plot([0.0001,0.001,0.01],n_list_train,label = 'train accuracy',marker = 'o')
plt.plot([0.0001,0.001,0.01],n_list_val,label = 'validation accuracy',marker = 'o')
plt.xlabel('C')
plt.ylabel('accuracy')
plt.legend(loc='upper right')
plt.title('Logistic Regression')
plt.show()

In [ ]:
model_log = LogisticRegression(C = 0.001)
model_log.fit(X_train,y_train)
pred_train = model_log.predict(X_train)
pred_val = model_log.predict(X_val)
val_score = accuracy_score(y_val,pred_val)
train_score = accuracy_score(y_train,pred_train)
print('train score is {}'.format(train_score))
print('validation score is {}'.format(val_score))
print('confusion matrix is \n {}'.format(confusion_matrix(y_val,pred_val)))

In [ ]:
df_test = pd.read_csv('../input/customer-churn-prediction-with-notebook/Churn Test Dataset.csv')
df_test = df_test[columns_for_test]
df_test.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
s = df_test.dtypes == 'object'
object_cols = s[s].index
for col in object_cols:
    df_test[col] = encoder.fit_transform(df_test[col])
y_test = df_test['Churn Indicator']
df_test.drop('Churn Indicator',axis = 1,inplace = True)
df_test.astype(float)
df_test = scaler.fit_transform(df_test)

In [ ]:
pred = model.predict(df_test)
score = accuracy_score(y_test.astype(float),pred)
print(score)
print(confusion_matrix(y_test, pred))

In [ ]:
pred = tree_model.predict(df_test)
score = accuracy_score(y_test.astype(float),pred)
print(score)
print(confusion_matrix(y_test, pred))

In [ ]:

pred = model_log.predict(df_test)
score = accuracy_score(y_test.astype(float),pred)
print(score)
print(confusion_matrix(y_test, pred))

> best model Random forest with validation accuracy-approx. 96% better choice for a balanced accuracy 
> if target is to only decrease FNR than Logistic regression will be better option 